# CNN from scratch

This notebook is intended for CIFAR-10 object recognition using CNN from scratch.

First, some imports.

In [1]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Activation, add
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import RMSprop
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as ktf
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook

Using TensorFlow backend.


Then, read the data, and normalize them.

In [12]:
# read data using Keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# check data size
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# normalize them to 0 mean and unit variance
train_data = ImageDataGenerator(samplewise_center=True,
                          samplewise_std_normalization=True,
                          featurewise_center=False,
                          featurewise_std_normalization=False)
train_data.fit(x_train)
test_data = ImageDataGenerator(samplewise_center=True,
                          samplewise_std_normalization=True,
                          featurewise_center=False,
                          featurewise_std_normalization=False)
test_data.fit(x_test)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


# SimpleCNN

Here we use a simple CNN model and train it from scratch.

In [14]:
# model architecture
# [batchnorm-Conv-Conv-maxpool]x2 - [dense]x2 - [softmax]
ktf.clear_session()
simple_CNN = Sequential()
simple_CNN.add(BatchNormalization(input_shape=(32, 32, 3)))
simple_CNN.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (16,16,32)

simple_CNN.add(BatchNormalization())
simple_CNN.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (8,8,64)

simple_CNN.add(BatchNormalization())
simple_CNN.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (4,4,128)

simple_CNN.add(BatchNormalization())
simple_CNN.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (2,2,256)

simple_CNN.add(Flatten())
simple_CNN.add(Dense(512, activation='relu'))
simple_CNN.add(Dropout(0.5))
simple_CNN.add(Dense(10, activation='softmax'))

# set loss and optimizer
rmsprop = RMSprop(lr=0.0005, decay=0.99)
simple_CNN.compile(loss='sparse_categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

# train the model
checkpoint = ModelCheckpoint('../models/simpleCNN_{epoch:02d}-{loss:.4f}.h5',
                             monitor='loss',
                             save_best_only=True)
earlystop = EarlyStopping(min_delta=0.0001, patience=3)

simple_CNN.fit_generator(generator=train_data.flow(x_train, y_train, batch_size=32),
                         steps_per_epoch=len(x_train)/32,
                         initial_epoch=0,
                         epochs=10,
                         callbacks=[],
                         validation_data=(test_data.flow(x_test, y_test, batch_size=32)),
                         validation_steps=len(x_test)/32)

Epoch 1/10
1562/1562 [============================>.] - ETA: 0s - loss: 2.1701 - acc: 0.1984

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,32,32]
	 [[Node: conv2d_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](batch_normalization_1/cond/Merge, conv2d_1/kernel/read)]]
	 [[Node: metrics/acc/Mean/_139 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_464_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv2d_1/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr

# Finetuning

# Create submission